In [1]:

import pandas as pd
import numpy as np
import re
import string
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns


In [13]:
# CONFIGURATION
INPUT_FILES = {
    'train': '/content/train_prepared.csv',
    'valid': '/content/valid_prepared.csv',
    'test': '/content/test_prepared.csv'
}


In [3]:

OUTPUT_FILES = {
    'train': 'train_cleaned.csv',
    'valid': 'valid_cleaned.csv',
    'test': 'test_cleaned.csv'
}


In [4]:

# Preprocessing options
CONFIG = {
    'lowercase': True,
    'remove_urls': True,
    'remove_mentions': True,
    'remove_hashtags': False,  
    'remove_numbers': True,
    'remove_punctuation': True,
    'remove_extra_spaces': True,
    'remove_emojis': True,
    'remove_duplicates': True,
    'remove_short_texts': True,  
    'min_text_length': 5,
    'normalize_repeated_chars': True,  
}


In [5]:

# Indonesian stopwords
INDONESIAN_STOPWORDS = set([
    'ada', 'adalah', 'adanya', 'adapun', 'agak', 'agaknya', 'agar', 'akan',
    'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku', 'akulah', 'amat',
    'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa',
    'apaan', 'apabila', 'apakah', 'apalagi', 'apatah', 'artinya', 'asal',
    'asalkan', 'atas', 'atau', 'ataukah', 'ataupun', 'awal', 'awalnya',
    'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi',
    'bagian', 'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan',
    'balik', 'banyak', 'bapak', 'baru', 'bawah', 'beberapa', 'begini',
    'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah',
    'begitupun', 'bekerja', 'belakang', 'belakangan', 'belum', 'belumlah',
    'benar', 'benarkah', 'benarlah', 'berada', 'berakhir', 'berakhirlah',
    'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti',
    'berawal', 'berbagai', 'berdatangan', 'beri', 'berikan', 'berikut',
    'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak',
    'berkeinginan', 'berkenaan', 'berlainan', 'berlalu', 'berlangsung',
    'berlebihan', 'bermacam', 'bermacam-macam', 'bermaksud', 'bermula',
    'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap', 'bertanya',
    'bertanya-tanya', 'berturut', 'berturut-turut', 'bertutur', 'berujar',
    'berupa', 'besar', 'betul', 'betulkah', 'biasa', 'biasanya', 'bila',
    'bilakah', 'bisa', 'bisakah', 'boleh', 'bolehkah', 'bolehlah', 'buat',
    'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan', 'bung', 'cara',
    'caranya', 'cukup', 'cukupkah', 'cukuplah', 'cuma', 'dahulu', 'dalam',
    'dan', 'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi', 'demikian',
    'demikianlah', 'dengan', 'depan', 'di', 'dia', 'diakhiri', 'diakhirinya',
    'dialah', 'diantara', 'diantaranya', 'diberi', 'diberikan', 'diberikannya',
    'dibuat', 'dibuatnya', 'didapat', 'didatangkan', 'digunakan', 'diibaratkan',
    'diibaratkannya', 'diingat', 'diingatkan', 'diinginkan', 'dijawab',
    'dijelaskan', 'dijelaskannya', 'dikatakan', 'dikatakannya', 'dikehendaki',
    'diketahui', 'diketahuinya', 'dikira', 'dilakukan', 'dilalui', 'dilihat',
    'dimaksud', 'dimaksudkan', 'dimaksudkannya', 'dimaksudnya', 'diminta',
    'dimintai', 'dimisalkan', 'dimulai', 'dimulailah', 'dimulainya', 'dimungkinkan',
    'dini', 'dipastikan', 'diperbuat', 'diperbuatnya', 'dipergunakan',
    'diperkirakan', 'diperlihatkan', 'diperlukan', 'diperlukannya',
    'dipersoalkan', 'dipertanyakan', 'dipunyai', 'diri', 'dirinya', 'disampaikan',
    'disebut', 'disebutkan', 'disebutkannya', 'disini', 'disinilah', 'ditambahkan',
    'ditandaskan', 'ditanya', 'ditanyai', 'ditanyakan', 'ditegaskan', 'ditujukan',
    'ditunjuk', 'ditunjuki', 'ditunjukkan', 'ditunjukkannya', 'ditunjuknya',
    'dituturkan', 'dituturkannya', 'diucapkan', 'diucapkannya', 'diungkapkan',
    'dong', 'dua', 'dulu', 'empat', 'enggak', 'enggaknya', 'entah', 'entahlah',
    'guna', 'gunakan', 'hal', 'hampir', 'hanya', 'hanyalah', 'hari', 'harus',
    'haruskah', 'haruslah', 'harusnya', 'hendak', 'hendaklah', 'hendaknya',
    'hingga', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu', 'ikut',
    'ingat', 'ingat-ingat', 'ingin', 'inginkah', 'inginkan', 'ini', 'inikah',
    'inilah', 'itu', 'itukah', 'itulah', 'jadi', 'jadilah', 'jadinya', 'jangan',
    'jangankan', 'janganlah', 'jauh', 'jawab', 'jawaban', 'jawabnya', 'jelas',
    'jelaskan', 'jelaslah', 'jelasnya', 'jika', 'jikalau', 'juga', 'jumlah',
    'jumlahnya', 'justru', 'kala', 'kalau', 'kalaulah', 'kalaupun', 'kalian',
    'kami', 'kamilah', 'kamu', 'kamulah', 'kan', 'kapan', 'kapankah', 'kapanpun',
    'karena', 'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya',
    'ke', 'keadaan', 'kebetulan', 'kecil', 'kedua', 'keduanya', 'keinginan',
    'kelamaan', 'kelihatan', 'kelihatannya', 'kelima', 'keluar', 'kembali',
    'kemudian', 'kemungkinan', 'kemungkinannya', 'kenapa', 'kepada', 'kepadanya',
    'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika',
    'khususnya', 'kini', 'kinilah', 'kira', 'kira-kira', 'kiranya', 'kita',
    'kitalah', 'kok', 'kurang', 'lagi', 'lagian', 'lah', 'lain', 'lainnya',
    'lalu', 'lama', 'lamanya', 'lanjut', 'lanjutnya', 'lebih', 'lewat', 'lima',
    'luar', 'lah', 'maka', 'makanya', 'makin', 'malah', 'malahan', 'mampu',
    'mampukah', 'mana', 'manakala', 'manalagi', 'masa', 'masalah', 'masalahnya',
    'masih', 'masihkah', 'masing', 'masing-masing', 'mau', 'maupun', 'melainkan',
    'melakukan', 'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan',
    'memberi', 'memberikan', 'membuat', 'memerlukan', 'memihak', 'meminta',
    'memintakan', 'memisalkan', 'memperbuat', 'mempergunakan', 'memperkirakan',
    'memperlihatkan', 'mempersiapkan', 'mempersoalkan', 'mempertanyakan',
    'mempunyai', 'memulai', 'memungkinkan', 'menaiki', 'menambahkan', 'menandaskan',
    'menanti', 'menanti-nanti', 'menantikan', 'menanya', 'menanyai', 'menanyakan',
    'mendapat', 'mendapatkan', 'mendatang', 'mendatangi', 'mendatangkan', 'menegaskan',
    'mengakhiri', 'mengapa', 'mengatakan', 'mengatakannya', 'mengenai', 'mengerjakan',
    'mengetahui', 'menggunakan', 'menghendaki', 'mengibaratkan', 'mengibaratkannya',
    'mengingat', 'mengingatkan', 'menginginkan', 'mengira', 'mengucapkan',
    'mengucapkannya', 'mengungkapkan', 'menjadi', 'menjawab', 'menjelaskan',
    'menuju', 'menunjuk', 'menunjuki', 'menunjukkan', 'menunjuknya', 'menurut',
    'menuturkan', 'menyampaikan', 'menyangkut', 'menyatakan', 'menyebutkan',
    'menyeluruh', 'menyiapkan', 'merasa', 'mereka', 'merekalah', 'merupakan',
    'meski', 'meskipun', 'meyakini', 'meyakinkan', 'minta', 'mirip', 'misal',
    'misalkan', 'misalnya', 'mula', 'mulai', 'mulailah', 'mulanya', 'mungkin',
    'mungkinkah', 'nah', 'naik', 'namun', 'nanti', 'nantinya', 'nyaris', 'nyatanya',
    'oleh', 'olehnya', 'pada', 'padahal', 'padanya', 'pak', 'paling', 'panjang',
    'pantas', 'para', 'pasti', 'pastilah', 'penting', 'pentingnya', 'per',
    'percuma', 'perlu', 'perlukah', 'perlunya', 'pernah', 'persoalan', 'pertama',
    'pertama-tama', 'pertanyaan', 'pertanyakan', 'pihak', 'pihaknya', 'pukul',
    'pula', 'pun', 'punya', 'rasa', 'rasanya', 'rata', 'rupanya', 'saat', 'saatnya',
    'saja', 'sajalah', 'saling', 'sama', 'sama-sama', 'sambil', 'sampai', 'sampai-sampai',
    'sampaikan', 'sana', 'sangat', 'sangatlah', 'saya', 'sayalah', 'se', 'sebab',
    'sebabnya', 'sebagai', 'sebagaimana', 'sebagainya', 'sebagian', 'sebaik',
    'sebaik-baiknya', 'sebaiknya', 'sebaliknya', 'sebanyak', 'sebegini', 'sebegitu',
    'sebelum', 'sebelumnya', 'sebenarnya', 'seberapa', 'sebesar', 'sebetulnya',
    'sebisanya', 'sebuah', 'sebut', 'sebutlah', 'sebutnya', 'secara', 'secukupnya',
    'sedang', 'sedangkan', 'sedemikian', 'sedikit', 'sedikitnya', 'seenaknya',
    'segala', 'segalanya', 'segera', 'seharusnya', 'sehingga', 'seingat', 'sejak',
    'sejauh', 'sejenak', 'sejumlah', 'sekadar', 'sekadarnya', 'sekali', 'sekali-kali',
    'sekalian', 'sekaligus', 'sekalipun', 'sekarang', 'sekarang', 'sekecil', 'seketika',
    'sekiranya', 'sekitar', 'sekitarnya', 'sekurang-kurangnya', 'sekurangnya',
    'sela', 'selain', 'selaku', 'selalu', 'selama', 'selama-lamanya', 'selamanya',
    'selanjutnya', 'seluruh', 'seluruhnya', 'semacam', 'semakin', 'semampu', 'semampunya',
    'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya', 'sementara', 'semisal',
    'semisalnya', 'sempat', 'semua', 'semuanya', 'semula', 'sendiri', 'sendirian',
    'sendirinya', 'seolah', 'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya',
    'sepantasnyalah', 'seperlunya', 'seperti', 'sepertinya', 'sepihak', 'sering',
    'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai', 'sesegera',
    'sesekali', 'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya',
    'setelah', 'setelahnya', 'setempat', 'setengah', 'seterusnya', 'setiap',
    'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya', 'setinggi', 'seusai',
    'sewaktu', 'siap', 'siapa', 'siapakah', 'siapapun', 'sini', 'sinilah', 'soal',
    'soalnya', 'suatu', 'sudah', 'sudahkah', 'sudahlah', 'supaya', 'tadi', 'tadinya',
    'tahu', 'tahun', 'tak', 'tambah', 'tambahnya', 'tampak', 'tampaknya', 'tandas',
    'tandasnya', 'tanpa', 'tanya', 'tanyakan', 'tanyanya', 'tapi', 'tegas', 'tegasnya',
    'telah', 'tempat', 'tengah', 'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat',
    'terakhir', 'terasa', 'terbanyak', 'terdahulu', 'terdapat', 'terdiri', 'terhadap',
    'terhadapnya', 'teringat', 'teringat-ingat', 'terjadi', 'terjadilah', 'terjadinya',
    'terkira', 'terlalu', 'terlebih', 'terlihat', 'termasuk', 'ternyata', 'tersampaikan',
    'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'terus', 'terutama', 'tetap',
    'tetapi', 'tiap', 'tiba', 'tiba-tiba', 'tidak', 'tidakkah', 'tidaklah', 'tiga',
    'tinggi', 'toh', 'tunjuk', 'turut', 'tutur', 'tuturnya', 'ucap', 'ucapnya',
    'ujar', 'ujarnya', 'umum', 'umumnya', 'ungkap', 'ungkapnya', 'untuk', 'usah',
    'usai', 'waduh', 'wah', 'wahai', 'waktu', 'waktunya', 'walau', 'walaupun',
    'wong', 'yaitu', 'yakin', 'yakni', 'yang'
])


In [6]:
# CLEANING FUNCTIONS
class TextCleaner:
    """Comprehensive text cleaning class"""

    def __init__(self, config):
        self.config = config
        self.emoji_pattern = re.compile(
            "["
            "\U0001F600-\U0001F64F"  # emoticons
            "\U0001F300-\U0001F5FF"  # symbols & pictographs
            "\U0001F680-\U0001F6FF"  # transport & map symbols
            "\U0001F1E0-\U0001F1FF"  # flags (iOS)
            "\U00002702-\U000027B0"
            "\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE
        )

    def clean_text(self, text):
        """Apply all cleaning steps to text"""
        if pd.isna(text) or not isinstance(text, str):
            return ""

        original_text = text

        # 1. Remove URLs
        if self.config['remove_urls']:
            text = re.sub(r'http\S+|www\.\S+', '', text)

        # 2. Remove mentions
        if self.config['remove_mentions']:
            text = re.sub(r'@\w+', '', text)

        # 3. Handle hashtags
        if self.config['remove_hashtags']:
            text = re.sub(r'#\w+', '', text)
        else:
            # Keep hashtag word but remove # symbol
            text = re.sub(r'#(\w+)', r'\1', text)

        # 4. Remove emojis
        if self.config['remove_emojis']:
            text = self.emoji_pattern.sub(r'', text)

        # 5. Normalize repeated characters (looooove -> love)
        if self.config['normalize_repeated_chars']:
            text = re.sub(r'(.)\1{2,}', r'\1\1', text)

        # 6. Remove numbers
        if self.config['remove_numbers']:
            text = re.sub(r'\d+', '', text)

        # 7. Remove punctuation
        if self.config['remove_punctuation']:
            text = text.translate(str.maketrans('', '', string.punctuation))

        # 8. Convert to lowercase
        if self.config['lowercase']:
            text = text.lower()

        # 9. Remove extra spaces
        if self.config['remove_extra_spaces']:
            text = ' '.join(text.split())

        # 10. Strip leading/trailing whitespace
        text = text.strip()

        return text

    def should_keep_text(self, text):
        """Determine if text should be kept after cleaning"""
        if not text or len(text.strip()) == 0:
            return False

        if self.config['remove_short_texts']:
            if len(text) < self.config['min_text_length']:
                return False

        return True


In [7]:
# PREPROCESSING PIPELINE
def preprocess_dataset(df, cleaner, dataset_name='dataset'):
    """Preprocess entire dataset"""

    print(f"\nProcessing {dataset_name}...")
    print(f"  Original size: {len(df)} samples")

    # Store original for comparison
    df['original_text'] = df['text'].copy()

    # Clean all texts
    print("  Cleaning texts...")
    df['text'] = df['text'].apply(cleaner.clean_text)

    # Count cleaning effects
    unchanged = (df['text'] == df['original_text']).sum()
    changed = len(df) - unchanged
    empty_after_cleaning = (df['text'].str.len() == 0).sum()

    print(f"    Unchanged: {unchanged}")
    print(f"    Modified: {changed}")
    print(f"    Empty after cleaning: {empty_after_cleaning}")

    # Remove texts that don't meet criteria
    if CONFIG['remove_short_texts'] or CONFIG['remove_duplicates']:
        before_filter = len(df)

        # Filter by text validity
        df = df[df['text'].apply(cleaner.should_keep_text)]

        # Remove duplicates
        if CONFIG['remove_duplicates']:
            df = df.drop_duplicates(subset=['text'], keep='first')

        removed = before_filter - len(df)
        print(f"  Removed {removed} samples")
        print(f"  Final size: {len(df)} samples")

    # Drop the original_text column
    df = df.drop('original_text', axis=1)

    return df


In [8]:

def generate_comparison_report(original_dfs, cleaned_dfs):
    """Generate before/after comparison report"""

    print("\n" + "="*100)
    print("PREPROCESSING REPORT")
    print("="*100)

    for name in original_dfs.keys():
        orig = original_dfs[name]
        clean = cleaned_dfs[name]

        print(f"\n{name.upper()} SET:")
        print(f"  Samples: {len(orig)} → {len(clean)} ({len(clean)-len(orig):+d})")

        # Text length statistics
        orig_lengths = orig['text'].str.len()
        clean_lengths = clean['text'].str.len()

        print(f"  Avg text length: {orig_lengths.mean():.1f} → {clean_lengths.mean():.1f} chars")
        print(f"  Min text length: {orig_lengths.min()} → {clean_lengths.min()} chars")
        print(f"  Max text length: {orig_lengths.max()} → {clean_lengths.max()} chars")

        # Label distribution
        print(f"  Label distribution:")
        for label in sorted(orig['label'].unique()):
            orig_count = (orig['label'] == label).sum()
            clean_count = (clean['label'] == label).sum()
            print(f"    Label {label}: {orig_count} → {clean_count} ({clean_count-orig_count:+d})")

    print("\n" + "="*100)
    print("CLEANING CONFIGURATION:")
    print("="*100)
    for key, value in CONFIG.items():
        print(f"  {key}: {value}")


In [9]:

def visualize_changes(original_dfs, cleaned_dfs):
    """Create visualizations comparing before/after"""

    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    datasets = list(original_dfs.keys())

    for idx, name in enumerate(datasets):
        orig = original_dfs[name]
        clean = cleaned_dfs[name]

        # Text length distribution
        ax = axes[0, idx]
        ax.hist([orig['text'].str.len(), clean['text'].str.len()],
               bins=30, label=['Original', 'Cleaned'], alpha=0.7,
               color=['red', 'green'])
        ax.set_title(f'{name.upper()} - Text Length', fontweight='bold')
        ax.set_xlabel('Length (chars)')
        ax.set_ylabel('Frequency')
        ax.legend()
        ax.grid(alpha=0.3)

        # Label distribution
        ax = axes[1, idx]
        labels = sorted(orig['label'].unique())
        x = np.arange(len(labels))
        width = 0.35
        orig_counts = [sum(orig['label'] == l) for l in labels]
        clean_counts = [sum(clean['label'] == l) for l in labels]

        ax.bar(x - width/2, orig_counts, width, label='Original', color='red', alpha=0.7)
        ax.bar(x + width/2, clean_counts, width, label='Cleaned', color='green', alpha=0.7)
        ax.set_title(f'{name.upper()} - Label Distribution', fontweight='bold')
        ax.set_xlabel('Label')
        ax.set_ylabel('Count')
        ax.set_xticks(x)
        ax.set_xticklabels(labels)
        ax.legend()
        ax.grid(axis='y', alpha=0.3)

    plt.tight_layout()
    plt.savefig('preprocessing_comparison.png', dpi=300, bbox_inches='tight')
    print("\n✓ Saved visualization: preprocessing_comparison.png")
    plt.close()


In [10]:

def show_examples(original_dfs, cleaned_dfs, n=10):
    """Show example transformations"""

    print("\n" + "="*100)
    print("EXAMPLE TRANSFORMATIONS (First 10)")
    print("="*100)

    df_orig = original_dfs['train']
    df_clean = cleaned_dfs['train']

    # Merge to compare
    comparison = pd.DataFrame({
        'original': df_orig['text'].head(n).values,
        'cleaned': df_clean['text'].head(n).values
    })

    for idx, row in comparison.iterrows():
        if row['original'] != row['cleaned']:
            print(f"\n[Example {idx+1}]")
            print(f"  BEFORE: {row['original'][:100]}")
            print(f"  AFTER:  {row['cleaned'][:100]}")
            print(f"  Length: {len(row['original'])} → {len(row['cleaned'])} chars")




In [18]:
import pandas as pd

def run_preprocessing_pipeline():
    print("="*100)
    print("AUTOMATIC DATA PREPROCESSING & CLEANING")
    print("="*100)

    # Initialize cleaner
    cleaner = TextCleaner(CONFIG)

    # Load original data
    print("\nLoading original data...")
    original_dfs = {}
    for name, filepath in INPUT_FILES.items():
        try:
            df = pd.read_csv(filepath)
            original_dfs[name] = df
            print(f"  ✓ Loaded {name}: {len(df)} samples")
        except Exception as e:
            print(f"  ✗ Error loading {name}: {e}")
            return # This now correctly exits the function on failure

    # Store copies for comparison
    original_copies = {name: df.copy() for name, df in original_dfs.items()}

    # Process each dataset
    cleaned_dfs = {}
    for name, df in original_dfs.items():
        # Ensure preprocess_dataset is defined
        cleaned_dfs[name] = preprocess_dataset(df, cleaner, name)

    # Save cleaned data
    print("\n" + "="*100)
    print("SAVING CLEANED DATA")
    print("="*100)
    for name, df in cleaned_dfs.items():
        filepath = OUTPUT_FILES[name]
        df.to_csv(filepath, index=False)
        print(f"  ✓ Saved {name} to {filepath}")

    # Generate report and visuals
    generate_comparison_report(original_copies, cleaned_dfs)
    show_examples(original_copies, cleaned_dfs)
    visualize_changes(original_copies, cleaned_dfs)

    print("\n" + "="*100)
    print("✓ PREPROCESSING COMPLETE!")
    print("="*100)

    orig_total = sum(len(df) for df in original_copies.values())
    clean_total = sum(len(df) for df in cleaned_dfs.values())
    reduction = orig_total - clean_total

    print(f"\n  Total samples: {orig_total} → {clean_total} ({reduction:+d})")
    print(f"  Reduction: {(reduction/orig_total*100):.1f}%")

# Execute the pipeline
run_preprocessing_pipeline()


AUTOMATIC DATA PREPROCESSING & CLEANING

Loading original data...
  ✓ Loaded train: 5500 samples
  ✓ Loaded valid: 1100 samples
  ✓ Loaded test: 4400 samples

Processing train...
  Original size: 5500 samples
  Cleaning texts...
    Unchanged: 10
    Modified: 5490
    Empty after cleaning: 0
  Removed 1 samples
  Final size: 5499 samples

Processing valid...
  Original size: 1100 samples
  Cleaning texts...
    Unchanged: 1
    Modified: 1099
    Empty after cleaning: 0
  Removed 0 samples
  Final size: 1100 samples

Processing test...
  Original size: 4400 samples
  Cleaning texts...
    Unchanged: 8
    Modified: 4392
    Empty after cleaning: 0
  Removed 2 samples
  Final size: 4398 samples

SAVING CLEANED DATA
  ✓ Saved train to train_cleaned.csv
  ✓ Saved valid to valid_cleaned.csv
  ✓ Saved test to test_cleaned.csv

PREPROCESSING REPORT

TRAIN SET:
  Samples: 5500 → 5499 (-1)
  Avg text length: 152.6 → 148.2 chars
  Min text length: 7 → 5 chars
  Max text length: 562 → 549 chars